In [ ]:
# Example with TensorFlow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

# Charger le dataset iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)  # Caractéristiques
y = pd.Series(iris.target)

# Fractionner les données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mise à l'échelle (scaling)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertir les cibles en one-hot encoding
y_train_categorical = to_categorical(y_train, num_classes=3)
y_test_categorical = to_categorical(y_test, num_classes=3)

### Premier modèle : Deep Neural Network (DNN)
model_dnn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes dans Iris
])

# Compiler le modèle
model_dnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model_dnn.fit(X_train, y_train_categorical, epochs=50, batch_size=8, verbose=1)

# Évaluer le modèle
loss_dnn, accuracy_dnn = model_dnn.evaluate(X_test, y_test_categorical, verbose=0)
print("DNN Model Accuracy:", accuracy_dnn)

### Deuxième modèle : Convolutional Neural Network (CNN)
# Adapter les données pour CNN (ajouter une dimension pour correspondre au format attendu)
X_train_cnn = X_train[..., np.newaxis]
X_test_cnn = X_test[..., np.newaxis]

model_cnn = Sequential([
    Conv1D(32, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    Conv1D(16, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes dans Iris
])

# Compiler le modèle
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
model_cnn.fit(X_train_cnn, y_train_categorical, epochs=50, batch_size=8, verbose=1)

# Évaluer le modèle
loss_cnn, accuracy_cnn = model_cnn.evaluate(X_test_cnn, y_test_categorical, verbose=0)
print("CNN Model Accuracy:", accuracy_cnn)

# Initialisation de l'application Flask
app = Flask(__name__)

# Choix du modèle sélectionné
# Remplacez `model_dnn` par `model_cnn` pour utiliser l'autre modèle
selected_model = model_dnn  # Utilisez `model_dnn` ou `model_cnn`

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/predict', methods=['GET'])
def predict():
    try:
        # Récupérer les arguments (attendus sous forme de chaîne CSV)
        features = request.args.get('features')  # Exemple: "5.1,3.5,1.4,0.2"
        features = np.array(features.split(','), dtype=float).reshape(1, -1)

        # Mise à l'échelle des caractéristiques
        features = scaler.transform(features)

        # Adapter les caractéristiques pour CNN si nécessaire
        if selected_model == model_cnn:
            features = features[..., np.newaxis]

        # Effectuer la prédiction (probabilités pour chaque classe)
        probabilities = selected_model.predict(features)[0]

        # Préparer une réponse JSON plus sobre
        response = {
            'probabilities': [{'class': i, 'probability': round(float(prob), 5)}
                              for i, prob in enumerate(probabilities)]
        }
        return jsonify(response)
    except Exception as e:
        return jsonify({'error': str(e)})

# Lancer l'application Flask
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5003)


Epoch 1/50


c:\Users\Tristan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5720 - loss: 1.0141  
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8391 - loss: 0.8160 
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7902 - loss: 0.7021 
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8283 - loss: 0.5750 
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8634 - loss: 0.4225 
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8408 - loss: 0.3859 
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9174 - loss: 0.2766 
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8669 - loss: 0.3174 
Epoch 9/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8585 - loss: 0.3248 
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8805 - loss: 0.2544 
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8766 - loss: 0.2465 
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9662 - l

c:\Users\Tristan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4875 - loss: 1.0842  
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7327 - loss: 0.9909 
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8393 - loss: 0.8673 
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8024 - loss: 0.7543 
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9089 - loss: 0.5440 
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8965 - loss: 0.4579 
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8360 - loss: 0.4498 
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9244 - loss: 0.2858 
Epoch 9/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9066 - loss: 0.2972 
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9309 - loss: 0.2482 
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9129 - loss: 0.2362 
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9422 - l

...

c:\Users\Tristan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jan/2025 16:31:42] "GET /predict?features=5.1,3.5,1.4,0.2 HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


c:\Users\Tristan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jan/2025 16:31:46] "GET /predict?features=5.1,3.5,1.4,0.2 HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


c:\Users\Tristan\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jan/2025 16:31:52] "GET /predict?features=5.1,3.5,1.4,0.2 HTTP/1.1" 200 -
